# Создай базовую папку
hdfs dfs -mkdir /user

# Сделай под неё папку под своего пользователя
hdfs dfs -mkdir /user/hadoop

# Дай на неё права (если надо)
hdfs dfs -chown hadoop:hadoop /user/hadoop


In [16]:
metadata_file = '/data/metadata_cleaned.csv'
images_dir = '/data/images'

In [17]:
# Путь в HDFS
hdfs_base = '/user/hadoop/covid_dataset/images/'
hdfs_metadata = '/user/hadoop/metadata/metadata_cleaned.csv'


In [23]:
from hdfs import InsecureClient

client = InsecureClient('http://hadoop-namenode:9870', user='hadoop')
print(client.list('/user/hadoop/'))

['covid_dataset', 'metadata']


In [ ]:
import os
import pandas as pd
# Читаем метаданные
df = pd.read_csv(metadata_file)

# Пробегаем по строкам
for idx, row in df.iterrows():
    filename = row['filename']
    finding = row['finding'].strip().replace(' ', '_')  # убираем пробелы

    local_file_path = os.path.join(images_dir, filename)
    hdfs_dir = os.path.join(hdfs_base, finding)
    hdfs_file_path = os.path.join(hdfs_dir, filename)

    # Создаём папку, если не существует
    if not client.status(hdfs_dir, strict=False):
        client.makedirs(hdfs_dir)
        print(f'Создана папка {hdfs_dir} в HDFS')

    if not os.path.exists(local_file_path):
        print(f'Файл {local_file_path} не найден, пропускаем...')
        continue
    # Загружаем файл
    with open(local_file_path, 'rb') as f:
        client.write(hdfs_file_path, f, overwrite=True)
        print(f'Загружен {filename} в {hdfs_file_path}')


Создана папка /user/hadoop/covid_dataset/images/COVID-19 в HDFS
Загружен auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg в /user/hadoop/covid_dataset/images/COVID-19/auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
Загружен auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg в /user/hadoop/covid_dataset/images/COVID-19/auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
Загружен auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg в /user/hadoop/covid_dataset/images/COVID-19/auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
Загружен auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg в /user/hadoop/covid_dataset/images/COVID-19/auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
Загружен nejmc2001573_f1a.jpeg в /user/hadoop/covid_dataset/images/COVID-19/nejmc2001573_f1a.jpeg
Загружен nejmc2001573_f1b.jpeg в /user/hadoop/covid_da

In [21]:
# Загрузить в HDFS
client.upload(hdfs_metadata, metadata_file)


print("Файл успешно загружен в HDFS:", hdfs_metadata)

Файл успешно загружен в HDFS: /user/hadoop/metadata/metadata_cleaned.csv
